# Setup

Important: If you're running on a local machine, be sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md). This notebook includes only what's necessary to run in Colab.

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path1 = 'gdrive/My Drive/MLforphysicist'
root_path2 = 'gdrive/My\ Drive/MLforphysicist'  #change dir to your project folder

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import pandas as pd
import numpy as np

#Path to Test csv for prediction
csv_path = root_path1+"/image_dataset/test_labels.csv"
csv = pd.read_csv(csv_path)


#Add columns for the prediction
csv['Predicted_xmin'] = ""
csv['Predicted_ymin'] = ""
csv['Predicted_xmax'] = ""
csv['Predicted_ymax'] = ""
csv['Predicted_class'] = ""
csv['Predicted_score'] = ""

### Install

In [3]:
!pip install -U --pre tensorflow=="2.*"
!pip install tf_slim

     |████████████████████████████████| 320.4MB 56kB/s 
     |████████████████████████████████| 460kB 29.8MB/s 
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0
     |████████████████████████████████| 358kB 3.3MB/s 


Make sure you have `pycocotools` installed

In [4]:
!pip install pycocotools

Get `tensorflow/models` or `cd` to parent directory of the repository.

In [5]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 2798, done.
remote: Counting objects: 100% (2798/2798), done.
remote: Compressing objects: 100% (2435/2435), done.
remote: Total 2798 (delta 567), reused 1406 (delta 327), pack-reused 0
Receiving objects: 100% (2798/2798), 57.73 MiB | 31.38 MiB/s, done.
Resolving deltas: 100% (567/567), done.


Compile protobufs and install the object_detection package

In [6]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

object_detection/protos/input_reader.proto: warning: Import object_detection/protos/image_resizer.proto but not used.


In [7]:
%%bash 
cd models/research
pip install .

Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1304063 sha256=661d3679496f1ca3d0db8f5e762bb5a6604e2a9b1b145ff1ea440fa82a0d4c6e
  Stored in directory: /tmp/pip-ephem-wheel-cache-j7zz4tyb/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
Successfully built object-detection


### Imports

In [8]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

Import the object detection module.

In [9]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

Patches:

In [10]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

# Model preparation 

## Variables

Any model exported using the `export_inference_graph.py` tool can be loaded here simply by changing the path.

By default we use an "SSD with Mobilenet" model here. See the [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md) for a list of other models that can be run out-of-the-box with varying speeds and accuracies.

## Loader

In [11]:
def load_model(model_name):

  model_dir = root_path1+"/models/"+model_name+"/saved_model"
  print(model_dir)

  model = tf.saved_model.load(str(model_dir))

  return model

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [12]:
# List of the strings that is used to add correct label for each box.
#PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
PATH_TO_LABELS = root_path1+"/data/label_map.txt"
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

For the sake of simplicity we will test on 2 images:

# Detection

Load an object detection model:

In [13]:
model_name = 'saved_model_v1'
detection_model = load_model(model_name)

gdrive/My Drive/MLforphysicist/models/saved_model_v1/saved_model


Check the model's input signature, it expects a batch of 3-color images of type uint8:

In [14]:
print(detection_model.signatures['serving_default'].inputs)

[<tf.Tensor 'input_tensor:0' shape=(1, None, None, 3) dtype=uint8>, <tf.Tensor 'unknown:0' shape=() dtype=resource>, <tf.Tensor 'unknown_0:0' shape=() dtype=resource>, <tf.Tensor 'unknown_1:0' shape=() dtype=resource>, <tf.Tensor 'unknown_2:0' shape=() dtype=resource>, <tf.Tensor 'unknown_3:0' shape=() dtype=resource>, <tf.Tensor 'unknown_4:0' shape=() dtype=resource>, <tf.Tensor 'unknown_5:0' shape=() dtype=resource>, <tf.Tensor 'unknown_6:0' shape=() dtype=resource>, <tf.Tensor 'unknown_7:0' shape=() dtype=resource>, <tf.Tensor 'unknown_8:0' shape=() dtype=resource>, <tf.Tensor 'unknown_9:0' shape=() dtype=resource>, <tf.Tensor 'unknown_10:0' shape=() dtype=resource>, <tf.Tensor 'unknown_11:0' shape=() dtype=resource>, <tf.Tensor 'unknown_12:0' shape=() dtype=resource>, <tf.Tensor 'unknown_13:0' shape=() dtype=resource>, <tf.Tensor 'unknown_14:0' shape=() dtype=resource>, <tf.Tensor 'unknown_15:0' shape=() dtype=resource>, <tf.Tensor 'unknown_16:0' shape=() dtype=resource>, <tf.Tenso

And returns several outputs:

In [15]:
detection_model.signatures['serving_default'].output_dtypes

{'detection_anchor_indices': tf.float32,
 'detection_boxes': tf.float32,
 'detection_classes': tf.float32,
 'detection_multiclass_scores': tf.float32,
 'detection_scores': tf.float32,
 'num_detections': tf.float32,
 'raw_detection_boxes': tf.float32,
 'raw_detection_scores': tf.float32}

In [16]:
detection_model.signatures['serving_default'].output_shapes

{'detection_anchor_indices': TensorShape([1, 100]),
 'detection_boxes': TensorShape([1, 100, 4]),
 'detection_classes': TensorShape([1, 100]),
 'detection_multiclass_scores': TensorShape([1, 100, 8]),
 'detection_scores': TensorShape([1, 100]),
 'num_detections': TensorShape([1]),
 'raw_detection_boxes': TensorShape([1, 300, 4]),
 'raw_detection_scores': TensorShape([1, 300, 8])}

Add a wrapper function to call the model, and cleanup the outputs:

In [17]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

Run it on each test image and show the results:

In [18]:
def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = np.array(Image.open(image_path))
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

  display(Image.fromarray(image_np))
  return output_dict

In [19]:
def mse_box(table):
  mse_xmax = (table['xmax']-table['Predicted_xmax'])**2
  mse_xmin = (table['xmin']-table['Predicted_xmin'])**2
  mse_ymin = (table['ymin']-table['Predicted_ymin'])**2
  mse_ymax = (table['ymax']-table['Predicted_ymax'])**2
  return (mse_xmax+mse_xmin+mse_ymax+mse_ymin)/4

In [20]:
csv_new = csv.copy()
for i in range(len(csv_new["filename"])):
  print("Predicting image",i+1,"/",len(csv_new["filename"]))
  im_path = root_path1+'/image_dataset/images/'+csv_new["filename"][i]
  im = Image.open(im_path)
  w,h = im.size
  image_np = np.array(im)
  output_dict = run_inference_for_single_image(detection_model,image_np)
  csv_new["Predicted_xmin"][i] = int(output_dict["detection_boxes"][0][1]*w)
  csv_new["Predicted_ymin"][i] = int(output_dict["detection_boxes"][0][0]*h)
  csv_new["Predicted_xmax"][i] = int(output_dict["detection_boxes"][0][3]*w)
  csv_new["Predicted_ymax"][i] = int(output_dict["detection_boxes"][0][2]*h)
  csv_new["Predicted_class"][i] = category_index[output_dict["detection_classes"][0]]['name']
  csv_new["Predicted_score"][i] = output_dict["detection_scores"][0]

print("Saving to csv")
csv_new.to_csv(root_path1+"/prediction/""predicted"+model_name[5:]+".csv")

Predicting image 1 / 500


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by Inte

Predicting image 2 / 500
Predicting image 3 / 500
Predicting image 4 / 500
Predicting image 5 / 500
Predicting image 6 / 500
Predicting image 7 / 500
Predicting image 8 / 500
Predicting image 9 / 500
Predicting image 10 / 500
Predicting image 11 / 500
Predicting image 12 / 500
Predicting image 13 / 500
Predicting image 14 / 500
Predicting image 15 / 500
Predicting image 16 / 500
Predicting image 17 / 500
Predicting image 18 / 500
Predicting image 19 / 500
Predicting image 20 / 500
Predicting image 21 / 500
Predicting image 22 / 500
Predicting image 23 / 500
Predicting image 24 / 500
Predicting image 25 / 500
Predicting image 26 / 500
Predicting image 27 / 500
Predicting image 28 / 500
Predicting image 29 / 500
Predicting image 30 / 500
Predicting image 31 / 500
Predicting image 32 / 500
Predicting image 33 / 500
Predicting image 34 / 500
Predicting image 35 / 500
Predicting image 36 / 500
Predicting image 37 / 500
Predicting image 38 / 500
Predicting image 39 / 500
Predicting image 40 

In [21]:
class_count = 0
wrong_label = []
for i in range(len(csv_new["class"])):
  if csv_new["class"][i] == csv_new["Predicted_class"][i]:
    class_count += 1
  else:
    wrong_label.append([csv_new["class"][i],csv_new["Predicted_class"][i]])

wrong_label.sort()
wrong_dict={}
for w in wrong_label:
  if w[0] not in wrong_dict:
    wrong_dict[w[0]] = 1
  else:
    wrong_dict[w[0]] += 1

print("Wrong assigned labels:")
for k in wrong_dict:
  print(k,":",wrong_dict[k])

print()
print("Right classes:",class_count/len(csv_new["class"])*100,"%")

Wrong assigned labels:
A : 59
C : 10
F : 10
H : 43
ID3 : 23
R : 1

Right classes: 70.8 %


In [22]:
output_dict = show_inference(detection_model, root_path1+'/image_dataset/images/'+csv["filename"][10])

print('Anzahl Detektionen:',output_dict['num_detections'])
print("Detektionen mit über 50% nur anzeigen")
for i in range(len(output_dict['detection_scores'])):
  if output_dict['detection_scores'][i] > 0.5:
    print('-'*50)
    print("Itteration:",i)
    print()
    print("Label:",category_index[output_dict['detection_classes'][i]]['name'])
    print("Genauigkeit von",output_dict['detection_scores'][i]*100,"%")
    print("Box:",output_dict['detection_boxes'][i])

Output hidden; open in https://colab.research.google.com to view.